In [ ]:
%pip -q install --upgrade ultralytics==8.3.199 pillow opencv-python tqdm

from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.4 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
data_yaml = """
path: /content/drive/MyDrive/censorly-dataset/Blood
train: images/train
val: images/validation
test: images/test
names: [blood]
task: segment
"""

with open("/content/drive/MyDrive/censorly-dataset/Blood/data.yaml", "w") as f:
    f.write(data_yaml)

print("data.yaml oluşturuldu ✅")


data.yaml oluşturuldu ✅


In [ ]:
from ultralytics import YOLO

# Model yolunu belirt
model_path = "/content/drive/MyDrive/censorly-training/blood_highres/best.pt"
model = YOLO(model_path)

# Test set yolunu data.yaml üzerinden alarak test splitini ölç
metrics = model.val(
    data="/content/drive/MyDrive/censorly-dataset/Blood/data.yaml",
    split="test",        # sadece test setini kullan
    imgsz=896,           # eğitimde kullandığın çözünürlük neyse onu gir (ör: 896)
    conf=0.45
)

print(metrics)  # precision, recall, mAP50, mAP50-95 vs.


Ultralytics 8.3.70 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11s-seg summary (fused): 265 layers, 10,067,203 parameters, 0 gradients, 32.8 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 13.8MB/s]
val: Scanning /content/drive/MyDrive/censorly-dataset/Blood/labels/test.cache... 333 images, 2 backgrounds, 0 corrupt: 100%|██████████| 333/333 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:09<00:00,  2.11it/s]


                   all        333       1431      0.804      0.487      0.664      0.564      0.803      0.486      0.663       0.48
Speed: 0.8ms preprocess, 4.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/segment/val
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78bf683b8aa0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.

In [ ]:
import cv2, json
import numpy as np
from pathlib import Path
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/censorly-training/blood_highres/best.pt")
test_dir = "/content/drive/MyDrive/censorly-dataset/Blood/images/test"

# 1) Test üzerinde tahmin al (txt/json kaydıyla)
res = model.predict(source=test_dir, imgsz=896, conf=0.20, iou=0.45, save_txt=True, save=True, retina_masks=True, verbose=False)

# 2) GT ve Pred poligonlarını oku, centroid∈GT ise TP say
#   Not: YOLO-seg label formatında her satır: cls x1 y1 x2 y2 ... (normalize koord.)
#   Basit yaklaşım: pred poligonunun centroid'ini hesapla; pointPolygonTest ile GT poligonunda mı bak
def poly_centroid(pts):
    # pts: Nx2 numpy (x,y) normalized
    # centroid (Shoelace method)
    x = pts[:,0]; y = pts[:,1]
    a = np.dot(x, np.roll(y,-1)) - np.dot(y, np.roll(x,-1))
    c_x = np.sum((x + np.roll(x,-1))*(x*np.roll(y,-1)-y*np.roll(x,-1))) / (3*(a+1e-9))
    c_y = np.sum((y + np.roll(y,-1))*(x*np.roll(y,-1)-y*np.roll(x,-1))) / (3*(a+1e-9))
    return np.array([c_x, c_y])

def read_yolo_seg(file, img_w, img_h):
    polys = []
    for line in open(file, "r"):
        parts = line.strip().split()
        if len(parts) < 6: continue
        cls = int(parts[0])
        coords = np.array(list(map(float, parts[1:])))
        xy = coords.reshape(-1,2)
        px = np.stack([xy[:,0]*img_w, xy[:,1]*img_h], axis=1)  # denormalize
        polys.append((cls, px.astype(np.float32)))
    return polys

# Basit skorlayıcı:
from glob import glob
import os

tp=0; fp=0; fn=0
img_paths = sorted(glob(os.path.join(test_dir, "*.*")))
for img_path in img_paths:
    img = cv2.imread(img_path)
    h,w = img.shape[:2]
    stem = Path(img_path).stem

    gt_file = Path(test_dir).with_name("..").resolve()  # images/test -> labels/test
    gt_file = (Path(str(gt_file))/ "labels" / "test" / f"{stem}.txt").resolve()
    pred_file = Path(model.predictor.save_dir)/"labels"/f"{stem}.txt"

    gts = read_yolo_seg(str(gt_file), w, h) if gt_file.exists() else []
    preds = read_yolo_seg(str(pred_file), w, h) if pred_file.exists() else []

    matched = set()
    for _, ppoly in preds:
        c = poly_centroid(ppoly)
        hit = False
        for gi,(gcls, gpoly) in enumerate(gts):
            # Nokta GT poligonunun içinde mi?
            inside = cv2.pointPolygonTest(gpoly, (float(c[0]), float(c[1])), False) >= 0
            if inside:
                hit = True
                matched.add(gi)
                break
        if hit: tp += 1
        else:   fp += 1

    fn += max(0, len(gts) - len(matched))

precision = tp/(tp+fp+1e-9)
recall    = tp/(tp+fn+1e-9)
f1        = 2*precision*recall/(precision+recall+1e-9)
print(f"[Centroid-in-GT]  P={precision:.3f}  R={recall:.3f}  F1={f1:.3f}")


Results saved to /content/runs/segment/predict
327 labels saved to /content/runs/segment/predict/labels
[Centroid-in-GT]  P=0.853  R=0.616  F1=0.715


In [ ]:
# === PATHS ===
DATA = "/content/drive/MyDrive/censorly-dataset/Blood/data.yaml"  # seg dataset
BASE = "/content/drive/MyDrive/best_models/blood_best.pt"  # mevcut iyi seg ağırlığın
PROJ = "/content/drive/MyDrive/censorly-training/blood_ft"               # yeni çıktı klasörü
NAME_VAL = "val_sweep_v1"      # validasyon çıktıları
NAME_FT  = "hnm_pack_v2"       # (opsiyonel) yeni fine-tune adı

# Kontrol (dosyalar var mı?)
import os, sys
for p in [DATA, BASE]:
    assert os.path.exists(p), f"Yol bulunamadı: {p}"
print("Yollar OK ✅")


Yollar OK ✅


In [ ]:
DATA = "/content/drive/MyDrive/censorly-dataset/Blood/data.yaml"

# v1 FT çıktın (yukarıda sweep yaptığın model)
BASE_V1 = "/content/drive/MyDrive/censorly-training/blood_ft/hnm_pack_v1/weights/best.pt"

# dataset train/val klasörleri (negatif oversampling için kullanacağız)
IMG_TR = "/content/drive/MyDrive/censorly-dataset/Blood/images/train"
LBL_TR = "/content/drive/MyDrive/censorly-dataset/Blood/labels/train"

PROJ = "/content/drive/MyDrive/censorly-training/blood_ft"
print("OK")


OK


In [ ]:
from ultralytics import YOLO

model = YOLO(BASE_V1)  # v1 best'ten devam
overrides = dict(
    task="segment",
    data=DATA,
    epochs=15,
    imgsz=960,
    batch=16,
    workers=4,            # pin_memory hatasında 0 yapabilirsin
    patience=7,
    lr0=0.0006, lrf=0.01, # düşük LR = daha temkinli sınır
    freeze=10,            # backbone'un ilk katmanlarını dondur
    # video benzetimli augment (hafif)
    hsv_h=0.012, hsv_s=0.55, hsv_v=0.28,
    degrees=5.0, translate=0.05, scale=0.50, shear=2.0, perspective=0.001,
    flipud=0.0, fliplr=0.5,
    mosaic=0.8, mixup=0.05, copy_paste=0.0,
    # FP'yi yumuşatmaya yardımcı
    label_smoothing=0.03,
    cls=0.85,
    optimizer="AdamW",    # daha düzgün genel geçerlik
    weight_decay=0.0005,
    cache=True,
    project=PROJ, name="hnm_pack_v1_headFT"
)
res = model.train(**overrides)
print("Yeni ağırlık:", f"{PROJ}/hnm_pack_v1_headFT/weights/best.pt")


New https://pypi.org/project/ultralytics/8.3.203 available 😃 Update with 'pip install -U ultralytics'
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in the future.
Ultralytics 8.3.199 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.85, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/censorly-dataset/Blood/data.yaml, degrees=5.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.012, hsv_s=0.55, hsv_v=0.28, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0006, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0

In [ ]:
import os, shutil, glob
from pathlib import Path

IMG_TR = Path(IMG_TR); LBL_TR = Path(LBL_TR)
dup_factor = 2  # 2x öneri (gerekirse 3 yapabilirsin)

made = 0
for lbl in LBL_TR.glob("*.txt"):
    # boş label = tam negatif
    if lbl.stat().st_size == 0:
        img_candidates = [IMG_TR / (lbl.stem + ext) for ext in [".jpg",".jpeg",".png"]]
        img = next((p for p in img_candidates if p.exists()), None)
        if not img:
            continue
        for k in range(1, dup_factor+1):
            img_new = IMG_TR / f"{lbl.stem}__dup{k}{img.suffix}"
            lbl_new = LBL_TR / f"{lbl.stem}__dup{k}.txt"
            if not img_new.exists():
                shutil.copy2(img, img_new)
                shutil.copy2(lbl, lbl_new)  # yine boş .txt
                made += 1
print("Eklenen negatif kopya:", made)


Eklenen negatif kopya: 834


In [ ]:
from ultralytics import YOLO
BASE_HEAD = f"{PROJ}/hnm_pack_v1_headFT/weights/best.pt"  # 1. adımdaki ağırlık

model = YOLO(BASE_HEAD)
res = model.train(
    task="segment", data=DATA,
    epochs=10, imgsz=960, batch=16, workers=4, patience=5,
    lr0=0.0005, lrf=0.01, freeze=10,
    hsv_h=0.010, hsv_s=0.50, hsv_v=0.28,
    mosaic=0.8, mixup=0.0, copy_paste=0.0,
    label_smoothing=0.03, cls=0.85,
    optimizer="AdamW", weight_decay=0.0005,
    cache=True, project=PROJ, name="hnm_pack_v1_headFT_negOS"
)
print("Yeni ağırlık:", f"{PROJ}/hnm_pack_v1_headFT_negOS/weights/best.pt")


New https://pypi.org/project/ultralytics/8.3.203 available 😃 Update with 'pip install -U ultralytics'
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in the future.
Ultralytics 8.3.199 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.85, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/censorly-dataset/Blood/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.01, hsv_s=0.5, hsv_v=0.28, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, 

In [ ]:
import cv2, os, glob, numpy as np
from pathlib import Path

IMG_TR = Path("/content/drive/MyDrive/censorly-dataset/Blood/images/train")
LBL_TR = Path("/content/drive/MyDrive/censorly-dataset/Blood/labels/train")  # YOLOv8-seg polygon format
OUTI = IMG_TR; OUTL = LBL_TR  # aynı yere kaydet (isim sonuna __cf ekle)

def poly_to_mask(h, w, poly):
    mask = np.zeros((h,w), np.uint8)
    pts = np.array(poly, dtype=np.int32).reshape(-1,2)
    cv2.fillPoly(mask, [pts], 255); return mask

def read_yolo_seg_label(p):
    polys=[]
    for line in open(p, "r", encoding="utf-8"):
        arr = list(map(float, line.strip().split()))
        if len(arr) >= 7:  # cls + x1 y1 x2 y2 ...
            xy = np.array(arr[1:], dtype=np.float32).reshape(-1,2)  # normalized
            polys.append(xy)
    return polys

made=0
for imgp in glob.glob(str(IMG_TR/"*.jpg")) + glob.glob(str(IMG_TR/"*.png")):
    stem = Path(imgp).stem
    lblp = LBL_TR/f"{stem}.txt"
    if not lblp.exists() or os.path.getsize(lblp)==0:
        continue  # pozitif değilse atla
    im = cv2.imread(imgp)
    h,w = im.shape[:2]
    polys_n = read_yolo_seg_label(lblp)
    if not polys_n:
        continue
    # birleşik maske
    mask = np.zeros((h,w), np.uint8)
    for xy in polys_n:
        poly = np.column_stack([xy[:,0]*w, xy[:,1]*h])
        cv2.fillPoly(mask, [poly.astype(np.int32)], 255)
    # inpaint
    dst = cv2.inpaint(im, mask, 5, cv2.INPAINT_TELEA)
    out_img = OUTI/f"{stem}__cf.jpg"
    out_lbl = OUTL/f"{stem}__cf.txt"
    if not out_img.exists():
        cv2.imwrite(str(out_img), dst)
        open(out_lbl, "w").close()  # BOŞ label: tam-negatif
        made += 1
print("Üretilen counterfactual negatif:", made)


Üretilen counterfactual negatif: 2268


In [ ]:
# === 0.2 DENGELEME (sağlam sürüm) ===
from pathlib import Path
import random, glob, os

IMG_DIR = Path("/content/drive/MyDrive/censorly-dataset/Blood/images/train")
LBL_DIR = Path("/content/drive/MyDrive/censorly-dataset/Blood/labels/train")

OUT_DIR = Path("/content/drive/MyDrive/censorly-dataset/Blood")
OUT_DIR.mkdir(parents=True, exist_ok=True) 
out_list = OUT_DIR / "train_balanced.txt"

imgs = sorted(
    glob.glob(str(IMG_DIR/"*.jpg")) +
    glob.glob(str(IMG_DIR/"*.png")) +
    glob.glob(str(IMG_DIR/"*.jpeg"))
)

pos, neg = [], []
for ip in imgs:
    stem = Path(ip).stem
    lp = LBL_DIR / f"{stem}.txt"
    if not lp.exists():
        continue
    if os.path.getsize(lp) > 0:
        pos.append(ip)
    else:
        neg.append(ip)

random.shuffle(neg)
target_neg = int(len(pos) * 0.55)  # ~%35 background hedefi
sel = pos + neg[:target_neg]
random.shuffle(sel)

out_list.write_text("\n".join(sel), encoding="utf-8")

print(f"✅ yazıldı: {out_list}")
print(f"pozitif: {len(pos)} | boş: {len(neg)} | kullanılan boş: {target_neg} | toplam train: {len(sel)}")


✅ yazıldı: /content/drive/MyDrive/censorly-dataset/Blood/train_balanced.txt
pozitif: 2268 | boş: 3519 | kullanılan boş: 1247 | toplam train: 3515


In [ ]:
import yaml, shutil
SRC_YAML = "/content/drive/MyDrive/censorly-dataset/Blood/data.yaml"
DST_YAML = "/content/drive/MyDrive/censorly-dataset/Blood/data_bal.yaml"
with open(SRC_YAML, "r") as f:
    y = yaml.safe_load(f)
y["train"] = str(out_list)  # train'i balanced listeye çevir
with open(DST_YAML, "w") as f:
    yaml.safe_dump(y, f, sort_keys=False)
print("✅ kullanacağın yaml:", DST_YAML)


✅ kullanacağın yaml: /content/drive/MyDrive/censorly-dataset/Blood/data_bal.yaml


In [ ]:
from ultralytics import YOLO
DATA = "/content/drive/MyDrive/censorly-dataset/Blood/data_bal.yaml"  # train: balanced txt
BASE = "/content/drive/MyDrive/censorly-training/blood_ft/s_curr_stage1/weights/best.pt"  # en iyi 's' ağırlığın

m = YOLO(BASE)
m.train(task="segment", data=DATA,
        epochs=6, imgsz=960, batch=16, workers=0, patience=5,
        lr0=6e-4, lrf=0.01, freeze=10,
        # AUG: sakin
        hsv_h=0.012, hsv_s=0.45, hsv_v=0.25,
        degrees=3, translate=0.03, scale=0.35, shear=1.0, perspective=0.0005,
        fliplr=0.5, flipud=0.0, mosaic=0.5, mixup=0.0, copy_paste=0.0,
        # LOSS/NMS
        cls=0.50, kobj=1.20, box=7.5, dfl=1.5,
        # kesinlikle kapatalım:
        auto_augment=None, erasing=0.0,
        # diğer
        optimizer="AdamW", weight_decay=5e-4,
        project="/content/drive/MyDrive/censorly-training/blood_ft", name="s_calm_A")
A_BEST = "/content/drive/MyDrive/censorly-training/blood_ft/s_calm_A/weights/best.pt"


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
New https://pypi.org/project/ultralytics/8.3.203 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.199 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=None, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/censorly-dataset/Blood/data_bal.yaml, degrees=3, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=6, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.

In [ ]:
m = YOLO(A_BEST)
m.train(task="segment", data=DATA,
        epochs=18, imgsz=960, batch=16, workers=0, patience=8,
        lr0=5e-4, lrf=0.01, freeze=0, cos_lr=True,
        hsv_h=0.012, hsv_s=0.40, hsv_v=0.25,
        degrees=3, translate=0.03, scale=0.35, shear=1.0, perspective=0.0005,
        fliplr=0.5, flipud=0.0, mosaic=0.5, mixup=0.0, copy_paste=0.0,
        cls=0.50, kobj=1.20, box=7.5, dfl=1.5,
        auto_augment=None, erasing=0.0,
        optimizer="AdamW", weight_decay=5e-4,
        project="/content/drive/MyDrive/censorly-training/blood_ft", name="s_calm_B")


New https://pypi.org/project/ultralytics/8.3.203 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.199 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=None, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/censorly-dataset/Blood/data_bal.yaml, degrees=3, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=18, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.012, hsv_s=0.4, hsv_v=0.25, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.2, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/censorly-training/blood_ft/s_calm_A/weights

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f3465f0d1f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 